# Ground truth neurons biological features

* author: steeve.laquitaine@epfl.ch

Setup spikeinterf_ env .. 

In [7]:
import os
from tqdm import tqdm
import numpy as np
import pandas as pd
import neurom as nm  # ground truth circuit details
import bluepy as bp  # ground truth circuit details
import spikeinterface as si
from neurom.view import plotly_impl
import neurom as nm


# move to PROJECT PATH
PROJ_PATH = "/gpfs/bbp.cscs.ch/project/proj85/home/laquitai/spikebias/"
os.chdir(PROJ_PATH)
from src.nodes.utils import get_config
from src.nodes.load import load_campaign_params

cfg, param_conf = get_config("silico_neuropixels", "npx_evoked").values()
SORTING_TRUE_PATH = cfg["ground_truth"]["full"]["output"]
BLUECFG = cfg["dataeng"]["blueconfig"]

# PARAMETERS

# SOMA
SOMA = {
    "soma": nm.core.types.NeuriteType.soma,
}
SOMA_FEATURES = [
    "radius",
    "volume",
]

# OTHER NEURITES
NEURITES_FEATURES = [
    "total_length",
    "total_height",
    "total_area",
    "number_of_sections",
    "section_lengths",
]
NEURITES = {
    "apical": nm.core.types.NeuriteType.apical_dendrite,
    "basal": nm.core.types.NeuriteType.basal_dendrite,
    "axon": nm.core.types.NeuriteType.axon,
}

E_MORPH = cfg["metadata"]["cell_morphs"]

2024-12-12 16:19:51,689 - root - utils.py - get_config - INFO - Reading experiment config.


2024-12-12 16:19:51,899 - root - utils.py - get_config - INFO - Reading experiment config. - done


### Custom functions

In [3]:
def get_features(morph_paths, feature_set, neurite_set):
    """Get morphometric features for all morphologies passed"""
    features = {"mtype": [], "feature_name": [], "feature_val": [], "loc": []}
    for mtype, paths in tqdm(morph_paths.items(), desc="mtypes"):
        for path in tqdm(
            paths, desc="morphologies", miniters=len(paths) / 100, leave=False
        ):
            morph = nm.load_morphology(path)
            for neurite_name, neurite_id in neurite_set.items():
                for feature in feature_set:
                    features["mtype"].append(mtype)
                    feature_name = (
                        "avg_section_length"
                        if feature == "section_lengths"
                        else feature
                    )
                    features["feature_name"].append(feature_name)
                    feature_val = nm.get(feature, morph, neurite_type=neurite_id)
                    feature_val = (
                        np.mean(feature_val)
                        if feature == "section_lengths"
                        else feature_val
                    )
                    features["feature_val"].append(feature_val)
                    features["loc"].append(neurite_name)
    df = pd.DataFrame.from_dict(features)
    df.drop(
        df.loc[(df["mtype"] == "L4_SSC") & (df["loc"] == "apical")].index, inplace=True
    )  # clear L4_SSC apicals...
    return df


def get_morph_paths(circuit, unit_ids: list):

    # get the units' mtypes and morphologies
    df = circuit.cells.get(unit_ids, properties=["mtype", "morphology"])

    # this creates the morphology path
    # for each unit by prefixing with the
    # common path of all morphologies
    # circuit.config["morphologies"]
    # to the morphology of the
    # units retrieved above
    # e.g., ../fixed_ais_L23PC_20201210/ascii/dend-C220797A-P3_axon-sm110131a1-3_IN.. for unit 12165
    # e.g., ../fixed_ais_L23PC_20201210/ascii/dend-rat_20160908_E3_LH2_cell2_axon-mt for unit 16652
    df["morph_path"] = (
        df["morphology"]
        .apply(lambda row: os.path.join(circuit.config["morphologies"], "%s.asc" % row))
        .astype(str)
    )
    df = df.drop(columns=["mtype", "morphology"])
    return df


def get_radius(circuit, unit_ids):

    # get the file paths of the morphology
    morph_paths = get_morph_paths(circuit, unit_ids)
    morph_paths = morph_paths.values.flatten()

    # get unit soma radius
    soma_radius = []
    for m_i in morph_paths:
        soma_radius.append(nm.load_morphology(m_i).soma.radius)
    return soma_radius

### Soma radius

In [4]:
# load once
circuit = bp.Simulation(BLUECFG).circuit

# get ground truth unit ids
SortingTrue = si.load_extractor(SORTING_TRUE_PATH)

In [6]:
# (1m)
# get volume
radius = get_radius(circuit, SortingTrue.unit_ids)

# unit-test
assert len(radius) == len(SortingTrue.unit_ids)
print(radius)

[7.209234, 8.141124, 6.5572352, 8.992678, 7.209234, 7.175878, 8.559333, 8.210715, 8.992678, 7.174989, 8.422994, 9.94524, 7.610226, 8.820431, 7.5481052, 8.559333, 12.163964, 11.027754, 7.183243, 11.027754, 7.610226, 8.482362, 8.141124, 6.5572352, 7.9881673, 8.082553, 9.324264, 7.9724455, 11.027754, 7.183243, 8.559333, 7.174989, 7.047762, 7.275731, 11.96777, 8.31106, 7.9881673, 8.361107, 8.361107, 10.477856, 9.0545635, 6.594521, 8.637625, 10.477856, 7.610226, 8.31106, 7.9724455, 7.5481052, 7.086813, 9.324264, 8.31106, 8.409002, 7.275731, 7.67147, 8.361107, 9.324264, 8.801092, 7.174989, 8.361107, 6.8325815, 8.559333, 8.210715, 8.210715, 8.559333, 6.8325815, 7.9724455, 8.361107, 11.027754, 6.7640333, 6.5572352, 7.275731, 7.047762, 6.594521, 8.210715, 11.96777, 6.5572352, 11.027754, 7.5481052, 8.082553, 7.275731, 7.086813, 8.210715, 8.422994, 7.183243, 8.422994, 6.594521, 7.5481052, 6.7640333, 8.992678, 7.9881673, 8.210715, 7.9881673, 14.586808, 7.174989, 7.209234, 8.361107, 8.293179, 8.637

### Neurites features

In [9]:
# 11 min

# get neurite features for each unit
df_all = pd.DataFrame()
for u_i, unit_id in enumerate(SortingTrue.unit_ids):

    # get neurite features
    morph_paths = get_morph_paths(circuit, [unit_id])
    df = get_features(morph_paths, feature_set=NEURITES_FEATURES, neurite_set=NEURITES)
    df = df.apply(lambda x: x.replace("apical", "apical_dendrite"))
    df = df.apply(lambda x: x.replace("basal", "basal_dendrite"))
    df = df.rename({"loc": "neurite_feature"}, axis="columns")
    df["neurite_feature"] += "_" + df["feature_name"]
    df.index = df.neurite_feature.tolist()
    df = df.drop(columns=["feature_name", "mtype", "neurite_feature"])
    df = df.T
    df["morph_path"] = morph_paths.iloc[0][0]
    
    # record
    df_all = pd.concat([df_all, df])

# drop index    
df_all = df_all.reset_index().drop(columns=["index"])

# concatenate with soma radius info
morph_df = pd.DataFrame()
morph_df["unit_id"] = SortingTrue.unit_ids
morph_df["soma_radius"] = radius
df = pd.concat([morph_df, df_all], axis=1)
df.to_parquet(E_MORPH)


mtypes: 1it [00:00,  3.57it/s]
mtypes: 1it [00:00,  5.15it/s]
mtypes: 1it [00:00,  3.58it/s]
mtypes: 1it [00:00,  7.45it/s]
mtypes: 1it [00:00,  5.51it/s]
mtypes: 1it [00:00,  6.28it/s]
mtypes: 1it [00:00,  5.46it/s]
mtypes: 1it [00:00,  6.59it/s]
mtypes: 1it [00:00,  4.17it/s]
mtypes: 1it [00:00, 10.14it/s]
mtypes: 1it [00:00,  5.64it/s]
mtypes: 1it [00:00,  5.57it/s]
mtypes: 1it [00:00,  6.83it/s]
mtypes: 1it [00:00,  5.92it/s]
mtypes: 1it [00:00,  8.27it/s]
mtypes: 1it [00:00,  5.43it/s]
mtypes: 1it [00:00,  3.23it/s]
mtypes: 1it [00:00,  5.08it/s]
mtypes: 1it [00:00,  3.30it/s]
mtypes: 1it [00:00,  4.80it/s]
mtypes: 1it [00:00,  3.08it/s]
mtypes: 1it [00:00,  7.07it/s]
mtypes: 1it [00:00,  7.44it/s]
mtypes: 1it [00:00,  5.11it/s]
mtypes: 1it [00:00,  2.18it/s]
mtypes: 1it [00:00,  5.88it/s]
mtypes: 1it [00:00,  2.67it/s]
mtypes: 1it [00:00, 10.15it/s]
mtypes: 1it [00:00,  8.67it/s]
mtypes: 1it [00:00,  3.26it/s]
mtypes: 1it [00:00,  4.61it/s]
mtypes: 1it [00:00, 10.82it/s]
mtypes: 

# References

https://neurom.readthedocs.io/en/latest/features.html#